## My notebook for segmenting and clustering neighborhoods in Toronto

### Scrape Wikipedia page

Installing prerequisites

In [1]:
#importing required libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

In [2]:
#installing BeautifulSoup library to help scrape WikiPage (if not installed before)
!pip install beautifulsoup4
#installing lxml parser for BeautifulSoup to use (if not installed before)
!pip install lxml
#installing requests to work with URLs (if not installed before)
!pip install requests

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
#importing BeautifulSoup and requests
from bs4 import BeautifulSoup
import requests

Work with URL and load data to dataframe

In [4]:
#load WikiPage with list of PostalCodes for Toronto
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [5]:
#cut a table from page to work with
table = soup.find('table', class_='wikitable sortable')

Constructing dataframe

In [6]:
#create columns out of headers in table
column_names = []
for header in table.find_all('th'):
    column_names.append(header.text.strip())
#create new dataframe with extracted columns
dataframe = pd.DataFrame(columns=column_names)

In [7]:
#extract rows from table based on <tr> tag
rows = table.find_all('tr')
for row in rows[1:]: #skipping zero element as it's header
    cols = row.find_all('td')
    if cols[1].text.strip() == 'Not assigned': #skipping lines with not assigned borough
        continue
    elif cols[2].text.strip() == 'Not assigned': #if neighbourhood not assigned make it equal to borough
        dataframe = dataframe.append([{'Postcode':cols[0].text.strip(), 'Borough':cols[1].text.strip(), 'Neighbourhood':cols[1].text.strip()}], ignore_index=True)
    else:
        dataframe = dataframe.append([{'Postcode':cols[0].text.strip(), 'Borough':cols[1].text.strip(), 'Neighbourhood':cols[2].text.strip()}], ignore_index=True) 


In [8]:
dataframe.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Combining neighbourhoods with the same postcode and borough

In [9]:
dataframe = dataframe.groupby(['Postcode','Borough'], sort=False).agg(','.join)

In [10]:
dataframe.head()

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park


In [11]:
dataframe.shape

(103, 1)